In [1]:
import gym
from rich import print

## list all the atari gym environments
env_names = [env_spec.id for env_spec in gym.envs.registry.all()]
print( env_names )

[
    'CartPole-v0',
    'CartPole-v1',
    'MountainCar-v0',
    'MountainCarContinuous-v0',
    'Pendulum-v1',
    'Acrobot-v1',
    'LunarLander-v2',
    'LunarLanderContinuous-v2',
    'BipedalWalker-v3',
    'BipedalWalkerHardcore-v3',
    'CarRacing-v1',
    'Blackjack-v1',
    'FrozenLake-v1',
    'FrozenLake8x8-v1',
    'CliffWalking-v0',
    'Taxi-v3',
    'Reacher-v2',
    'Pusher-v2',
    'InvertedPendulum-v2',
    'InvertedDoublePendulum-v2',
    'HalfCheetah-v2',
    'HalfCheetah-v3',
    'Hopper-v2',
    'Hopper-v3',
    'Swimmer-v2',
    'Swimmer-v3',
    'Walker2d-v2',
    'Walker2d-v3',
    'Ant-v2',
    'Ant-v3',
    'Humanoid-v2',
    'Humanoid-v3',
    'HumanoidStandup-v2'
]

In [34]:
env_id = "CartPole-v1"
seed = 42
def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video:
            if idx == 0:
                env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env

    return thunk
envs = gym.vector.SyncVectorEnv(
        [make_env( env_id = env_id, 
                  seed = seed , idx = 42, capture_video = False , run_name = "start") for i in range(1)]

)


In [35]:
from cleanrl.ppo import Agent

model = Agent(
    envs = envs,
    quantize = True,
)

print( type( model  ))

print( model )

print( model.critic )

<class 'cleanrl.ppo.Agent'>

Agent(
  (critic): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
  (actor): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=2, bias=True)
  )
)

Sequential(
  (0): Linear(in_features=4, out_features=64, bias=True)
  (1): Tanh()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): Tanh()
  (4): Linear(in_features=64, out_features=1, bias=True)
)

In [36]:
print( model.actor.modules )
print( model.named_children())

<bound method Module.modules of Sequential(
  (0): Linear(in_features=4, out_features=64, bias=True)
  (1): Tanh()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): Tanh()
  (4): Linear(in_features=64, out_features=2, bias=True)
)>

<generator object Module.named_children at 0x7f3c217e0ed0>

In [37]:
for x in model.actor.named_children():
    print( x )

('0', Linear(in_features=4, out_features=64, bias=True))

('1', Tanh())

('2', Linear(in_features=64, out_features=64, bias=True))

('3', Tanh())

('4', Linear(in_features=64, out_features=2, bias=True))

In [47]:
import torch
import torch.nn as nn
layers = []

for x in model.actor.named_children():
    if  isinstance(x[1],nn.Linear):
            layers.append( [x[0] , str(int(x[0])+1) ] )

print( layers )
torch.ao.quantization.fuse_modules( model.actor, layers, inplace=True)

[['0', '1'], ['2', '3'], ['4', '5']]

AssertionError: did not find fuser method for: (<class 'torch.nn.modules.linear.Linear'>, <class 'torch.nn.modules.activation.Tanh'>) 